## Launch an Inference Server (vLLM) for the base Model

This step sets up a vLLM inference server to host your base model and exposes an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance like throughput, latency, and time-to-first-token. The performance benchmarks between the base and base model will be used later on.

**Goal**: Make the base model accessible via an API for performance evaluation.

**Output**: vLLM server running with the base model, ready to accept requests.

**Resources used** : 46GB L40S GPU x 1


In [ ]:
import os

from utils import generate, stream

In [ ]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

### vLLM config for single node

We will be using the configuration for a single-node, signle-GPU set up to launch a vLLM server for the base model. 

Run the following command in terminal to serve the base model using vLLM

**NOTE**: 
- If your resources cannot serve the two (compressed and base) models together, make sure to stop the vLLM server for the compressed model(if running) before starting the base model server or you will get an Out Of Memory(OOM) error.

- If your system can host the two models simutalnously, keep the ``--port`` parameter different for the base and compressed models.

- The configuration used to serve the base model and the compressed model (in the [Compressed.ipynb](Compressed.ipynb) notebook) is the same other than the model name.

- Make sure you are in the parent directory of base_model.
  
```
vllm serve \
  "base_model" \
  --host 127.0.0.1 \
  --port 8000 \
  --gpu-memory-utilization 0.6 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1 \
  --max-model-len 2048
```

Once the server starts, you will see something like this:

```INFO:     Started server process [166518]```\
```INFO:     Waiting for application startup.```\
```INFO:     Application startup complete.```


### A test run to see if the vLLM server is accessible
We use a helper function **generate** (defined in [utils.py](./utils.py)) to simplify sending requests to our locally-served VLLM model.

This function wraps the OpenAI-compatible Chat Completions API exposed by VLLM.

In [ ]:
# For non streaming results
response = generate(
    model="base_model",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
)
print(response)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy and organic compounds for nearly all living organisms.

The word "photosynthesis" comes from the Greek words "photo" (light) and "synthesis" (putting together). During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the process:

1. **Light absorption**: Plants absorb light energy from the sun through specialized pigments such as chlorophyll.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the atmosphere through small openings

In [ ]:
# For streaming results
res = ""
for chunk in stream(
    model="base_model",
    prompt="What is photosynthesis?",
    host="127.0.0.1",
    port=8000,
    api_key="empty",
    max_tokens=512,
):
    res += chunk
    print(chunk, end="", flush=True)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy and organic compounds for nearly all living organisms.

During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose (a type of sugar) and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the process:

1. **Light absorption**: Chlorophyll, a green pigment found in plant cells, absorbs light energy from the sun.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through small openings on their leaves called stomata.
4. **Light-dependent reactions**: Light energy is used to convert

In [ ]:
print(res)

Photosynthesis is a vital process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of glucose. This process is essential for life on Earth, as it provides the primary source of energy and organic compounds for nearly all living organisms.

During photosynthesis, plants use energy from sunlight, water, and carbon dioxide to produce glucose (a type of sugar) and oxygen. The overall equation for photosynthesis is:

6 CO2 (carbon dioxide) + 6 H2O (water) + light energy → C6H12O6 (glucose) + 6 O2 (oxygen)

Here's a simplified overview of the process:

1. **Light absorption**: Chlorophyll, a green pigment found in plant cells, absorbs light energy from the sun.
2. **Water absorption**: Plants absorb water from the soil through their roots.
3. **Carbon dioxide absorption**: Plants absorb carbon dioxide from the air through small openings on their leaves called stomata.
4. **Light-dependent reactions**: Light energy is used to convert

### Checking GPU vRAM
When loading the base model with the configuration defined above, the GPU memory usage is approximately **28.5 GB**, similar to what was observed for the compressed model. This might seem surprising because the base model is almost **twice the size** of the compressed model. This is because the ``--gpu-memory-utilization`` flag is set to ``0.6`` for both models, so in any case, vLLM is going to use 60% of the GPU memory. 

The memory usage can be broken down as follows:

- **Model Weights:** About **16 GB** is used to store the weights of the base model (compared to ~8 GB for the compressed model).  
- **Remaining GPU Memory (~12 GB):** Reserved for **runtime buffers, KV cache, and other GPU operations** required by vLLM.
